<center><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQXnrruP7guibF61c4gOESBR2VhhqYmEjKin0yY9lrhCak3hIbXI0QDSlObO74xcBkjMw&usqp=CAU"/></center>

# MAKE GRID - NOTEBOOK VERSION

<small>
🔎 The current interface for creating grids is too heavy to load when the code is run remotely. The aim is to create a notebook(s) that can fulfill the role of the "make_grid.py" code and also edit the mask and bathymetry.
<br>💡 This notebook is linked to the croco-ocean/croco_pytolls' [ISSUE #4](https://gitlab.inria.fr/croco-ocean/croco_pytools/-/issues/4) located on the **croco-ocean INRIA's gitlab**
<br>➡️ Notebook created by <a href="mailto:thibault.delahaye@ird.fr">Thibault Delahaye</a> and based on the existing codes of pycrocotool
<br>🌳 Developped on **notebook-grid-tools**'s branch, whose source is the main branch
</small>

## Table of contents 🔗

1. [Code presentation 📖](#📖INFOS)
2. [Requirements ⚙️](#⚙️Requirements)
3. [CODE BODY](#CODE_BODY)

### 📖INFOS

Further Information: [CROCO OCEAN WEBSITE](http://www.croco-ocean.org)
  
*This file is part of CROCOTOOLS* 🐊

🌐 Create a CROCO grid file  
In the current state the script can handle:  

| DATA 💾     | SOURCE | FILE EXTENSION    |
|------------|------------|---------------|
| **Topo/Bathy** | etopo (5,2,1), | *NetCDF*    |
|            | srtm30, gebco  |           |
| **Coastline**  | GSHHS          | *shapefile* |
 

➡️ *To add a new dataset you just have to go into Readers/topo_readers.py and
create a dictionnary with the names for lon,lat,topo in the dataset.
At this time it only handles 1D lon/la*  
➡️ *The script makes a grid with mercator projection centred at the equator
before rotating the sphere to put the middle of the grid in tra_lon/tra_lat
position.*  
➡️ *Then, it reads topo's dataset and apply the desired smoothing
The mask is generated using a shapefile (.shp. default is gshhs dataset)*  
➡️ *Smoothing uses fortran routines for better performance*


### ⚙️Requirements
➡️ Installing `ipympl` for Interactive Plots :

*To use interactive plotting features with the `%matplotlib widget` magic command in Jupyter Notebook or JupyterLab, you need to install the `ipympl` package. This package enables Matplotlib to render interactive plots using Jupyter widgets.*

 **Run the following command** in a code cell or in your notebook to install `ipympl`:
   ```bash
   pip install ipympl


=============================================================================================================================

# CODE_BODY

## Prerequisites

### Magic cells

In [11]:
"""
Magic cell dedicated to the activation of backend environments
"""
#Allows widget interactivity with user
%matplotlib widget    
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### DEPENDENCIES

In [14]:
import numpy as np
import os,sys
os.environ['ETS_TOOLKIT'] = 'wx'
sys.path.append("./Modules/")
sys.path.append("./Readers/")
sys.path.append("./Modules/graphicUI_tools/")
from main_window import *
import tools_make_grid

## USER's CHANGES

In [15]:
# Grid center [degree]
tra_lon =  15 # Longitude of the grid center 
tra_lat = -32 # Latitude of the grid center

# Grid size [km]
size_x = 1556
size_y = 1334

# Grid number of points 
# Note: grid resolution is grid size / number of points
nx = 39
ny = 40

# Grid rotation [degree]
rot = 0

# Smoothing parameters
# (see online documentation for more details)
hmin        = 50    # Minimum depth [m]
hmax        = 6000  # Maximum depth [m]
interp_rad  = 2     # Interpolation radius in number of points (usually between 2 and 8)
rfact       = 0.2   # Maximum r-fact to reach (the lower it is, the smoother it will be)    
smooth_meth = 'lsmooth' # Smoothing method ('smooth', 'lsmooth', 'lsmooth_legacy', 'lsmooth2', 'lsmooth1', 'cond_rx0_topo') 

# Topo/Bathy file
topofile = '../../DATASETS_CROCOTOOLS/Topo/etopo2.nc'

# Coastline file (for the mask)
shp_file = '../../DATASETS_CROCOTOOLS/gshhs/GSHHS_shp/i/GSHHS_i_L1.shp'

# Single Connect [Mask water not connected to the main water body]
sgl_connect=[False,20,20] # True or False, point indices inside the main water body

# Output grid file
output_file="../../CROCO_FILES/croco_grd.nc"
